In [ ]:
# https://towardsdatascience.com/training-rl-agents-in-stable-baselines3-is-easy-9d01be04c9db

In [1]:
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import A2C

2023-06-12 16:52:19.770949: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 16:52:19.830316: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 16:52:20.783024: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
env_id = "Breakout-v4"
gif_folder = "logfiles/gifs"
models_folder = "models"

In [3]:
# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
env = make_atari_env('BreakoutNoFrameskip-v4', n_envs=16)
# Frame-stacking with 4 frames
env = VecFrameStack(env, n_stack=4)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [4]:
model = A2C("CnnPolicy", env, verbose=1)
model.learn(log_interval=1E5, total_timesteps=5E6, progress_bar=True)
model.save(f"{models_folder}/a2c_cnn_vec16_{env_id}")

Using cuda device
Wrapping the env in a VecTransposeImage.


Output()

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [6]:
env = make_atari_env(env_id, n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [7]:
images = []
obs = env.reset()
img = env.render(mode="rgb_array")
for i in range(500):
# done = False
# while not done:
    images.append(img)
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    img = env.render(mode="rgb_array")

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:364: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


In [8]:
import imageio
import numpy as np

In [16]:
# fps = 60*4
# dur = len(images)/fps
dur = 50
imageio.mimsave(f"{env_id}_cnn_vec.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=dur)